<a href="https://colab.research.google.com/github/hselino/complaint_analysis/blob/main/sample_response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# classification metrics

In [3]:
# Load dataset
df = pd.read_csv("/content/drive/MyDrive/prj/segmented_class.csv")
df = df.drop(columns=["Unnamed: 0"], errors='ignore')

# Drop any rows with missing data
df = df.dropna()


In [ ]:
df.head()

,complaint_text,category
0,kendi homepoint finansal olarak kısmen ol cidd...,ödeme süreci sırasında sorun
1,equifax kredi raporum sahtekarlık uyarım var r...,raporunuzun uygunsuz kullanımı
2,son zaman aldatmaca işle para kimlik çal kişis...,borçlu borç alma denemeleri
3,dağ zirves financial tan kredi ön plan çık ilg...,borçlu servic iletişime geçilemiyor
4,arizona doğum tarih ss transunion llc tüket ci...,raporunuzdaki yanlış bilgiler


In [4]:
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['category'])


In [5]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['complaint_text'].tolist(),
    df['encoded_label'].tolist(),
    test_size=0.2,
    random_state=42
)

In [6]:
bert1_path = "/content/drive/MyDrive/prj/fine_tuned_model/class_model"
bert2_path = "/content/drive/MyDrive/prj/fine_tuned_model/berturk_model"
turna_path = "/content/drive/MyDrive/prj/fine_tuned_model/turna_model"
multi_path = "/content/drive/MyDrive/prj/fine_tuned_model/multilingual_model"

In [7]:
def evaluate_model(model_path, val_texts, val_labels):
    from transformers import AutoModelForSequenceClassification, AutoTokenizer
    import numpy as np
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model and tokenizer from the same path
    model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # Tokenize inputs
    inputs = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        preds = torch.argmax(probs, dim=1).cpu().numpy()

    true = np.array(val_labels)
    acc = accuracy_score(true, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(true, preds, average='weighted')

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [ ]:
model_paths = [bert1_path, bert2_path, turna_path, multi_path]

for path in model_paths:
    metrics = evaluate_model(path, val_texts, val_labels)
    print(f"📦 Metrics for {path}:")
    for k, v in metrics.items():
        print(f"{k.capitalize()}: {v:.4f}")
    print("—" * 40)


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


📦 Metrics for /content/drive/MyDrive/prj/fine_tuned_model/class_model:
Accuracy: 0.4975
Precision: 0.3917
Recall: 0.4975
F1: 0.4127
————————————————————————————————————————


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


📦 Metrics for /content/drive/MyDrive/prj/fine_tuned_model/berturk_model:
Accuracy: 0.0525
Precision: 0.0259
Recall: 0.0525
F1: 0.0300
————————————————————————————————————————


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


📦 Metrics for /content/drive/MyDrive/prj/fine_tuned_model/turna_model:
Accuracy: 0.1105
Precision: 0.0242
Recall: 0.1105
F1: 0.0361
————————————————————————————————————————
📦 Metrics for /content/drive/MyDrive/prj/fine_tuned_model/multilingual_model:
Accuracy: 0.0750
Precision: 0.0247
Recall: 0.0750
F1: 0.0365
————————————————————————————————————————


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# rouge score

In [ ]:
!pip install evaluate
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=af6b87b3153a546d35f09a78ee444282f03f893deea5302fa0f4276f4b1893bb
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [8]:
summ_df = pd.read_csv("/content/drive/MyDrive/prj/segmented_summ.csv")
summ_df = summ_df.dropna()
texts = summ_df['content'].tolist()
summaries = summ_df['abstract'].tolist()

In [ ]:
summ_df.head()

,abstract,content,title
0,şarkıcı tuğba özerk annesine canlı yayında yön...,tuğba özerk duy kal program anne günay kapani ...,tuğba özerkten deniz akkayaya 100 bin tllik dava
1,mhp lideri bahçeli cumhurbaşkanlığı hükümet si...,mhp genel başkan bahçe li manisa selendi ilçe ...,bahçeli cepheleşme keskinleşirse mhp buna tepk...
2,pentagon abd askerlerinin suriyede kürt birlik...,rusya suriye hava müdahale başla ardından abd ...,ypg ve abd omuz omuza
3,galatasarayda forvet transferinde halen somut ...,galatasa forvet transfer halen somut adım at s...,neredesin forvet
4,fenerbahçe teknik direktörü christoph daum isp...,ispanya marca gazete yayımla demecin daum güiz...,daum guiza mutsuz


In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

summ_path = "/content/drive/MyDrive/prj/fine_tuned_model/summarizer"

summarizer_model = AutoModelForSeq2SeqLM.from_pretrained(summ_path).to(device)
summarizer_tokenizer = AutoTokenizer.from_pretrained(summ_path)


In [10]:
from tqdm import tqdm

def generate_summaries(texts, model, tokenizer, max_len=128):
    model.eval()
    summaries = []

    for text in tqdm(texts, desc="Generating summaries"):
        try:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
            with torch.no_grad():
                output_ids = model.generate(**inputs, max_length=max_len, num_beams=4)
            summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            summaries.append(summary)
        except Exception as e:
            print(f"Error for input: {text[:30]}... -> {str(e)}")
            summaries.append("")  # or handle differently

    return summaries


In [11]:
generated = generate_summaries(texts[:10], summarizer_model, summarizer_tokenizer)

Generating summaries: 100%|██████████| 10/10 [01:20<00:00,  8.01s/it]


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

results = rouge.compute(predictions=generated, references=summaries[:10])

for metric, score in results.items():
    print(f"{metric}: {score:.4f}")


rouge1: 0.3744
rouge2: 0.2198
rougeL: 0.3383
rougeLsum: 0.3389


# response

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
import torch

summ_path = "/content/drive/MyDrive/prj/fine_tuned_model/summarizer"
bert1_path = "/content/drive/MyDrive/prj/fine_tuned_model/class_model"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

summarizer_model = AutoModelForSeq2SeqLM.from_pretrained(summ_path).to(device)
summarizer_tokenizer = AutoTokenizer.from_pretrained(summ_path)

classifier_model = AutoModelForSequenceClassification.from_pretrained(bert1_path).to(device)
classifier_tokenizer = AutoTokenizer.from_pretrained(bert1_path)

In [13]:
def summarize(text):
    inputs = summarizer_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    summary_ids = summarizer_model.generate(**inputs, max_length=128, num_beams=4)
    return summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def classify(text, label_encoder):
    inputs = classifier_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        logits = classifier_model(**inputs).logits
    predicted = torch.argmax(logits, dim=1).cpu().numpy()[0]
    return label_encoder.inverse_transform([predicted])[0]

In [17]:
complaint = "15 Mart 2025 tarihinde Adana Barajyolu Ziraat Bankası şube ATM'sinde 13 bin para yatırma işlemi gerçekleştirdim, ancak ATM para haznesine nakiti koyduğumda para haznesi kapandı ve kartımı geri iade etti, (normalde para sayıp ardından ekrana tutarın yansıması onayla kısmı gelmesi gerekti) yani paramı yuttu. Ortak ATM olduğu için işlemi QNB kartım ile gerçekleştirdim, Ziraat Bankası aradığımda kendi bankanı ara dedi. QNB bankası ise talep oluşturduklarını söyledi. Tam 15 gün geçti sonuçlanmadı, dönüş yapılmıyor dendi. Bu sefer Ziraat Barajyolu şubesine gittim tekrar, sert ve kırıcı üsluplarıyla karşılaştım. Bana kendi bankanın şube müdürlüğüne ulaşması lazım! Yazılı dilekçe ver o bankadan! Dedi. Zaman ayırıp tekrar gittim böyle bir işlem olmadığını işleyişinde bu şekilde olmadığını söylediler. Bugün 23. Gün alenen parama el konulduğunu düşünüyorum. Bu kadar işlek bir yerde her gün para sayılır fazla tutar bellidir. Bir talebe bu kadar süre dönülmemesi ve iş bilmemeleri şok edici. Gelişme olursa yazımı güncelleyeceğim"

summary = summarize(complaint)
category = classify(complaint, label_encoder)

prompt = f"""
Kullanıcıdan gelen şikayet: "{complaint}"
Şikayetin özeti: "{summary}"
Şikayet türü: {category}

Kullanıcıya Türkçe, kibar ve çözüm odaklı bir şekilde yanıt ver.
"""


In [14]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 3.1 MB/s eta 0:00:00


In [15]:
import os
from groq import Groq
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
app = Groq()

In [25]:
client = Groq(
    api_key=os.environ.get("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"),
)

# Uzunluklara göre max_tokens değerleri
lengths = {
    "Kısa": {"verbal": "Lütfen yanıtı kısa ve Türkçe olacak şekilde ver."},
    "Orta": {"verbal": "Lütfen yanıtı orta uzunlukta ve Türkçe bir hikâye olarak anlat."},
    "Uzun": {"verbal": "Lütfen uzun, detaylı ve Türkçe bir hikâye yaz."}
}


print(f"complaint: {complaint} \n")
print(f"summary: {summary} \n")
print(f"category: {category} \n")

for label, config in lengths.items():
    print(f"\n--- {label} Yanıt ---\n")

    full_prompt = f"{config['verbal']} {prompt}"

    response = client.chat.completions.create(
        messages=[
            {"role": "user", "content": full_prompt}
        ],
        model="llama3-70b-8192",

        temperature=0.7
    )

    print(response.choices[0].message.content)


complaint: 15 Mart 2025 tarihinde Adana Barajyolu Ziraat Bankası şube ATM'sinde 13 bin para yatırma işlemi gerçekleştirdim, ancak ATM para haznesine nakiti koyduğumda para haznesi kapandı ve kartımı geri iade etti, (normalde para sayıp ardından ekrana tutarın yansıması onayla kısmı gelmesi gerekti) yani paramı yuttu. Ortak ATM olduğu için işlemi QNB kartım ile gerçekleştirdim, Ziraat Bankası aradığımda kendi bankanı ara dedi. QNB bankası ise talep oluşturduklarını söyledi. Tam 15 gün geçti sonuçlanmadı, dönüş yapılmıyor dendi. Bu sefer Ziraat Barajyolu şubesine gittim tekrar, sert ve kırıcı üsluplarıyla karşılaştım. Bana kendi bankanın şube müdürlüğüne ulaşması lazım! Yazılı dilekçe ver o bankadan! Dedi. Zaman ayırıp tekrar gittim böyle bir işlem olmadığını işleyişinde bu şekilde olmadığını söylediler. Bugün 23. Gün alenen parama el konulduğunu düşünüyorum. Bu kadar işlek bir yerde her gün para sayılır fazla tutar bellidir. Bir talebe bu kadar süre dönülmemesi ve iş bilmemeleri şok edi